In [1]:
# ипрортируем библиотеки

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [24]:
photo_train = pd.read_csv('C://Users/alex1/Downloads/photo/train.tsv',sep='\t', header=None, names=['название_файла', 'правильный_текст'])
photo_test = pd.read_csv('C://Users/alex1/Downloads/photo/test.tsv',sep='\t', header=None, names=['название_файла', 'правильный_текст'])

photo_train = photo_train.fillna(0)
photo_train = photo_train.dropna()
photo_train = photo_train.dropna(axis=1)
print(photo_train.isnull().sum())

название_файла      0
правильный_текст    0
dtype: int64


In [30]:
photo_test = photo_test.fillna(0)
photo_test = photo_test.dropna()
photo_test = photo_test.dropna(axis=1)
print(photo_test.isnull().sum())

название_файла      0
правильный_текст    0
dtype: int64


In [5]:
photo_train

,название_файла,правильный_текст
0,aa1.png,Молдова
1,aa1007.png,продолжила борьбу
2,aa101.png,разработанные
3,aa1012.png,Плачи
4,aa1013.png,Гимны богам
...,...,...
72281,yob20539.png,Ответственность в/сл-
72282,yob20543.png,независимо
72283,yob20544.png,от воинского звания
72284,yob20545.png,воинской


In [4]:
photo_test


,название_файла,правильный_текст
0,test0.png,ибо
1,test1.png,осталось
2,test10.png,поле
3,test100.png,Оптическое
4,test1000.png,1 класса
...,...,...
1539,testtt658.png,г. Иркутска
1540,testtt659.png,"Андреевна,"
1541,testtt677.png,Юрьевна
1542,testtt692.png,девяносто)


In [27]:
import os
import cv2
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Функция для преобразования изображения в черно-белый формат и изменения размера
def preprocess_image(file_path):
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (28, 28))
    image = image.astype('float32') / 255.0
    return image

# Путь к папке с фотографиями
photo_folder = 'C://Users/alex1/Downloads/photo/train'

# Загрузка датафрейма
photo_train = photo_train
metki_ind = photo_train['правильный_текст'].value_counts()[photo_train['правильный_текст'].value_counts() >= 10].index
photo_train = photo_train[photo_train['правильный_текст'].isin(metki_ind)]
# Преобразование фотографий в черно-белый формат и изменение размера
photo_train['image'] = photo_train['название_файла'].apply(
    lambda x: preprocess_image(os.path.join(photo_folder, x)))

# Разбиение столбца 'правильный_текст' на классы
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
photo_train['class'] = photo_train['правильный_текст']

# Использование LabelEncoder для столбца 'правильный_текст'
label_encoder = LabelEncoder()
photo_train['class_encoded'] = label_encoder.fit_transform(photo_train['class'].astype(str))

# One-hot encoding
photo_train_encoded = pd.get_dummies(photo_train["class_encoded"])

# Добавить столбцы с закодированными значениями в датафрейм
photo_train = pd.concat([photo_train, photo_train_encoded], axis=1)
# Объединение оригинального датафрейма и one-hot encoded меток



In [31]:
photo_folder = 'C://Users/alex1/Downloads/photo/test'

# Загрузка датафрейма
photo_test = photo_test
metki_ind = photo_test['правильный_текст'].value_counts()[photo_test['правильный_текст'].value_counts() >= 1].index
photo_test = photo_test[photo_test['правильный_текст'].isin(metki_ind)]
# Преобразование фотографий в черно-белый формат и изменение размера
photo_test['image'] = photo_test['название_файла'].apply(
    lambda x: preprocess_image(os.path.join(photo_folder, x)))

# Разбиение столбца 'правильный_текст' на классы
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
photo_test['class'] = photo_test['правильный_текст']

# Использование LabelEncoder для столбца 'правильный_текст'
label_encoder = LabelEncoder()
photo_test['class_encoded'] = label_encoder.fit_transform(photo_test['class'].astype(str))

# One-hot encoding
photo_test_encoded = pd.get_dummies(photo_test["class_encoded"])

# Добавить столбцы с закодированными значениями в датафрейм
photo_test = pd.concat([photo_test, photo_test_encoded], axis=1)
# Объединение оригинального датафрейма и one-hot encoded меток


In [9]:
photo_train

,название_файла,правильный_текст,image,class,class_encoded,0,1,2,3,4,...,835,836,837,838,839,840,841,842,843,844
0,aa1.png,Молдова,"[[0.9529412, 0.9647059, 0.8039216, 0.9019608, ...",Молдова,64,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,aa1044.png,территории,"[[0.5764706, 0.4627451, 0.58431375, 0.5647059,...",территории,738,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,aa1046.png,нет,"[[0.6039216, 0.59607846, 0.6117647, 0.6039216,...",нет,423,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,aa1070.png,Египет,"[[0.2627451, 0.23529412, 0.2509804, 0.27058825...",Египет,34,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46,aa1082.png,нет,"[[0.65882355, 0.654902, 0.6509804, 0.6509804, ...",нет,423,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72253,yob20475.png,также,"[[0.6039216, 0.6117647, 0.6117647, 0.60784316,...",также,721,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72256,yob20481.png,службу,"[[0.5882353, 0.57254905, 0.5686275, 0.5686275,...",службу,665,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72266,yob20504.png,имеет,"[[0.5176471, 0.53333336, 0.5294118, 0.5254902,...",имеет,288,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72271,yob20516.png,воинской,"[[0.5882353, 0.5764706, 0.57254905, 0.52156866...",воинской,167,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
photo_test

,название_файла,правильный_текст,image,class,class_encoded,0,1,2,3,4,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
0,test0.png,ибо,"[[0.99215686, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99607...",ибо,685,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,test1.png,осталось,"[[0.68235296, 0.68235296, 0.6745098, 0.6784314...",осталось,885,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,test10.png,поле,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",поле,970,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,test100.png,Оптическое,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",Оптическое,307,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,test1000.png,1 класса,"[[0.85882354, 0.87058824, 0.87058824, 0.866666...",1 класса,23,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539,testtt658.png,г. Иркутска,"[[0.78039217, 0.92156863, 0.91764706, 0.403921...",г. Иркутска,558,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1540,testtt659.png,"Андреевна,","[[1.0, 1.0, 1.0, 0.99607843, 1.0, 0.99607843, ...","Андреевна,",135,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1541,testtt677.png,Юрьевна,"[[0.78431374, 0.7921569, 0.7921569, 0.7921569,...",Юрьевна,433,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1542,testtt692.png,девяносто),"[[1.0, 0.99607843, 1.0, 1.0, 1.0, 1.0, 1.0, 1....",девяносто),601,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
photo_test[['название_файла'	,'правильный_текст','class_encoded']]
df_photo_test = photo_test.copy()
df_photo_test = df_photo_test.dropna()
df_photo_test['class_encoded'].max()

1309

In [16]:
df
X_df_photo_test = list(df_photo_test['image'])
X_df_photo_test = np.asarray(X_df_photo_test)
y_df_photo_test = df_photo_test.drop(['название_файла',	'правильный_текст',	'image',	'class',	'class_encoded'],axis = 1)
y_df_photo_test = np.asarray(y_df_photo_test).astype('float32')

In [29]:
photo_train[['название_файла'	,'правильный_текст','class_encoded']]
df = photo_train.copy()
df = df.dropna()
df['class_encoded'].max()

844

In [12]:
df
X = list(df['image'])
X = np.asarray(X)
y = df.drop(['название_файла',	'правильный_текст',	'image',	'class',	'class_encoded'],axis = 1)
y = np.asarray(y).astype('float32')

In [17]:
len(X_df_photo_test)

1544

In [22]:
num_classes = 845
import keras
from keras import layers 
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [33]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X, y, epochs=10, batch_size=64)

Epoch 1/10
290/290 [==============================] - 7s 21ms/step - loss: 5.8610 - accuracy: 0.0621
Epoch 2/10
290/290 [==============================] - 6s 22ms/step - loss: 4.8210 - accuracy: 0.1572
Epoch 3/10
290/290 [==============================] - 6s 22ms/step - loss: 3.8593 - accuracy: 0.2625
Epoch 4/10
290/290 [==============================] - 6s 22ms/step - loss: 3.1991 - accuracy: 0.3506
Epoch 5/10
290/290 [==============================] - 6s 22ms/step - loss: 2.6962 - accuracy: 0.4304
Epoch 6/10
290/290 [==============================] - 6s 22ms/step - loss: 2.2925 - accuracy: 0.4958
Epoch 7/10
290/290 [==============================] - 6s 22ms/step - loss: 1.9658 - accuracy: 0.5569
Epoch 8/10
290/290 [==============================] - 6s 22ms/step - loss: 1.6797 - accuracy: 0.6106
Epoch 9/10
290/290 [==============================] - 7s 22ms/step - loss: 1.4374 - accuracy: 0.6577
Epoch 10/10
290/290 [==============================] - 7s 23ms/step - loss: 1.2184 - accura

In [34]:
test_loss, test_acc = model.evaluate(X_df_photo_test, y_df_photo_test)
test_acc

ValueError: in user code:

    File "C:\Users\alex1\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1972, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\alex1\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1956, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\alex1\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1944, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\alex1\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1852, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\alex1\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "C:\Us

In [35]:
test_loss

NameError: name 'test_loss' is not defined